<img src="./img/logoconvexbrancomini2.png"  align="right"/>

<!--
<img src="./img/logoconvexbrancomini2.png"  align="right"/>
-->
# Resource Allocation Problem

<!--
<img src="./img/logoboxverde.png" align="right"/>
-->
__by [Daniel Cinalli](http://www.cinalli.com.br)__ - DSc Artificial Intelligence

## Uncapacitated Facility Location - Problem #09



<br/><br/> 
## Notes:

* Coded in Python 3.x
* Using [Anaconda](https://www.anaconda.com/) is recommended
* Run the notebook `online` at [binder](https://mybinder.org/v2/gh/drcinalli/Artificial-Intelligence-and-Transformation/master) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/drcinalli/Artificial-Intelligence-and-Transformation/master)
<!-- * [nbviewer](https://nbviewer.jupyter.org/) allows you to switch the notebooks "slides" mode-->

<br> </br>
### Table of Contents

- [Problem](#prob)
- [Simplex](#simplex)
- [Random Heuristic](#random)
- [Lowest Shipping Cost (per Client) Heuristic](#lowShip)
- [Greatest Shipping Cost (per Client) Heuristic](#maxShip)
- [Lowest Shipping & Fixed Costs Heuristic](#lowShipFix)
- [Greatest Shipping & Fixed Costs Heuristic](#maxShipFix)

<br>
<br>

<a id='prob'></a>
## Problem #09

<br>
Facilities <br>
$|I| = 2000$ 
<br>
<br>
Clients <br>
$|J| = 2000$ 


<br> 
<br>


<a id='simplex'></a>
### Simplex (exact)



In [8]:
from itertools import product
from math import sqrt
import gurobipy as gp
from gurobipy import GRB
import time

# Get Clients and Facilities
def getFacilities_Clients(file_list):
    return int(file_list[0]), int(file_list[1])

# Get Facilities Fixed Costs
def getFacilities_Capacity_FixedCosts(file_list, num_facilities):
    shift = 2
    capacity = []
    cost = []
    
    #loop to get all i location costs
    for i in range(0,num_facilities*2,2):
        capacity.append(int(file_list[i+shift]))
        cost.append(int(file_list[i+1+shift].replace(".","")))
    
    return capacity, cost

# Get Facilities Fixed Costs
def getFacilities_STRCapacity_FixedCosts(file_list, num_facilities):
    shift = 2
    capacity = []
    cost = []
    
    #loop to get all i location costs
    for i in range(0,num_facilities*2,2):
        capacity.append(file_list[i+shift])
        cost.append(int(file_list[i+1+shift].replace(".","")))
    
    return capacity, cost


# Get Demand and Allocation Costs for j(customer) to each i(client)
def getClient_Demand_AllocationCosts(file_list, num_facilities, num_customers):
    shift = 2 + (num_facilities*2)
    demand = []
    allocation_cost = []
    
    #loop to get all j Clients 
    j=0
    for r in range(0,num_customers):
    
        #get demand
        demand.append(int(file_list[j+shift]))

        #loop to get all i location costs
        for i in range(0,num_facilities):
            allocation_cost.append(float(file_list[j+1+i+shift]))
            
        #fix j
        j += num_facilities+1
 
    
    return demand, allocation_cost

#Read File from OR datasets
fileName='datasets/MT1'
ORlist = []

with open(fileName, "r") as f:
    ORlist = f.read().split()
    
##### Sets and Indices #####
num_facilities, num_customers = getFacilities_Clients(ORlist)
capacity, fixed_cost = getFacilities_Capacity_FixedCosts(ORlist, num_facilities)
cartesian_prod = list(product(range(num_customers), range(num_facilities)))
# shipping costs
demand, alloc_cost = getClient_Demand_AllocationCosts(ORlist, num_facilities, num_customers)
shipping_cost = dict(zip(cartesian_prod, alloc_cost))



start = time.time()


# MIP  model formulation
#m = gp.Model('UFLP')


##### Decision Variable #####
#x = m.addVars(num_facilities, vtype=GRB.BINARY, name='x')
#y = m.addVars(cartesian_prod, ub=1, vtype=GRB.CONTINUOUS, name='y')

##### Constraints #####
#m.addConstrs((y[(c,f)] <= x[f] for c,f in cartesian_prod), name='Shipping')
#m.addConstrs((gp.quicksum(y[(c,f)] for f in range(num_facilities)) == 1 for c in range(num_customers)), name='Demand')

##### Objective Function #####
#m.setObjective(x.prod(fixed_cost)+y.prod(shipping_cost), GRB.MINIMIZE)

#m.Params.Method = 1
# Options are:-1=automatic, 0=primal simplex, 1=dual simplex, 2=barrier, 3=concurrent, 4=deterministic concurrent, 5=deterministic concurrent simplex

#m.optimize()


end = time.time()
print("TIME IS: ",end - start)

TIME IS:  0.00011873245239257812


In [9]:
# display optimal values of decision variables

#for facility in x.keys():
#    if (abs(x[facility].x) > 1e-6):
#        print(f"\nBuild a warehouse at location {facility + 1}.")

# Shipments from facilities to customers.

#for customer, facility in y.keys():
#    if (abs(y[customer, facility].x) > 1e-6):
#        print(f"\nClient {customer + 1} receives {round(100*y[customer, facility].x, 2)} % of its demand  from Warehouse {facility + 1} .")

#for v in m.getVars():
#    print(v.varname, v.x)

#print(f"\nOptimal total:", m.objVal)

#m.write('UFLP_09_Simplex.lp')

<br>
<br>

<a id='random'></a>
### Random Heuristic 


In [10]:
#very naive/simple
#for each client, choose randomly one of the Facilities available
import random


start = time.time()

result=[]
#choose the Facility for each customer
for i in range(num_customers):
    result.append((i,random.randint(0, num_facilities-1)))
   

#remove duplication of facilities in order to print properly
facs=[]
for i in result:
    facs.append(i[1])
#print(result)
#print("xxx")
#print (facs)
facs=list(set(facs))
    
#print("xxx")
print (facs)

#calculate the setup_cost
totalC=0
for w in facs:
    totalC += fixed_cost[w]
    #print(w)

#print(totalC)

#calculate the shipping cost
for i in result: 
    totalC += shipping_cost.get(i)
    #print (shipping_cost.get(i))
    
print(totalC)



end = time.time()
print("TIME IS: ",end - start)

#setup_cost
#cost_per_mile*compute_distance(customers[c], facilities[f]) 
#list(set(output))

[1, 3, 4, 5, 6, 7, 9, 10, 11, 14, 15, 17, 22, 23, 24, 25, 26, 27, 28, 29, 31, 36, 37, 38, 41, 42, 44, 45, 46, 47, 49, 51, 52, 56, 57, 59, 60, 64, 65, 70, 72, 73, 74, 76, 77, 78, 80, 81, 82, 83, 84, 86, 87, 88, 91, 92, 93, 94, 95, 96, 97, 100, 102, 103, 104, 105, 108, 111, 112, 113, 114, 115, 118, 119, 120, 123, 125, 126, 127, 131, 132, 134, 136, 138, 139, 140, 141, 143, 144, 145, 146, 148, 150, 151, 152, 153, 154, 155, 156, 158, 160, 161, 163, 168, 171, 172, 173, 174, 176, 177, 179, 180, 181, 184, 185, 186, 187, 190, 191, 193, 195, 196, 200, 202, 203, 204, 206, 207, 208, 211, 212, 213, 214, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 232, 235, 236, 237, 238, 239, 240, 242, 244, 245, 246, 248, 249, 250, 251, 252, 255, 256, 258, 259, 261, 262, 264, 265, 268, 269, 271, 272, 273, 274, 275, 276, 279, 280, 281, 283, 284, 286, 290, 291, 292, 295, 297, 299, 300, 302, 305, 306, 308, 309, 310, 311, 312, 313, 315, 316, 317, 318, 319, 320, 321, 322, 324, 325, 327, 33

In [11]:
for i in facs:
    print(f"\nBuild a warehouse at location {i + 1}.")

for i in result:
    print(f"\nClient {i[0] + 1} receives 100% of its demand  from Warehouse {i[1] + 1} .")

print(f"\nOptimal total:", totalC)


Build a warehouse at location 2.

Build a warehouse at location 4.

Build a warehouse at location 5.

Build a warehouse at location 6.

Build a warehouse at location 7.

Build a warehouse at location 8.

Build a warehouse at location 10.

Build a warehouse at location 11.

Build a warehouse at location 12.

Build a warehouse at location 15.

Build a warehouse at location 16.

Build a warehouse at location 18.

Build a warehouse at location 23.

Build a warehouse at location 24.

Build a warehouse at location 25.

Build a warehouse at location 26.

Build a warehouse at location 27.

Build a warehouse at location 28.

Build a warehouse at location 29.

Build a warehouse at location 30.

Build a warehouse at location 32.

Build a warehouse at location 37.

Build a warehouse at location 38.

Build a warehouse at location 39.

Build a warehouse at location 42.

Build a warehouse at location 43.

Build a warehouse at location 45.

Build a warehouse at location 46.

Build a warehouse at loca

In [12]:
len(facs)

1282

<br>
<br>

<a id='lowShip'></a>
### Lowest Shipping Cost (per client) Heuristic



In [13]:
#Get the lowest shipping cost for each Client


start = time.time()

#choose the lowest cost 
path={}
for i in range(num_customers):
    aux={}
    aux_key=()
    for j in range(num_facilities):

        #empty list for the Client
        if not aux:
            aux[(i,j)] = shipping_cost.get((i,j)) 
            aux_key = ((i,j)) 
            #print(aux)
            #print(aux_key)
        elif aux[aux_key]>shipping_cost.get((i,j)):
            #print("....")
            #print (i[0],j[0])
            #print(aux[aux_key])
            #print(shipping_cost.get((i[0],j[0])))
            #print ("  ")
            aux.pop(aux_key)
            aux[(i,j)] = shipping_cost.get((i,j))             
            aux_key = ((i,j))
            
        #print(aux_key)
    #print ("xxxxxxx")
    path.update(aux)

#print(path)    

#calculate the setup_cost
facs=[]
for i in path:
    facs.append(i[1])
#print (facs)
facs=list(set(facs))
print (facs)

totalC=0
for w in facs:
    totalC += fixed_cost[w]
    #print(w)

#print(totalC)

#calculate the shipping cost
for i in path.values():
    #print(i)
    totalC += i 
    
print(totalC)
print (len(facs))


end = time.time()
print("TIME IS: ",end - start)

[3, 4, 5, 6, 8, 9, 10, 12, 16, 17, 18, 19, 22, 24, 25, 26, 27, 29, 30, 31, 33, 38, 39, 40, 41, 42, 43, 44, 45, 48, 49, 50, 51, 52, 54, 55, 56, 58, 59, 60, 61, 62, 64, 65, 66, 67, 69, 70, 71, 72, 73, 74, 75, 76, 77, 79, 80, 81, 82, 83, 86, 88, 89, 90, 92, 93, 94, 96, 97, 99, 100, 101, 102, 104, 106, 107, 109, 110, 111, 112, 113, 114, 115, 117, 120, 121, 123, 125, 126, 127, 129, 130, 132, 133, 135, 137, 139, 140, 141, 142, 144, 145, 148, 150, 151, 152, 153, 154, 155, 156, 157, 161, 162, 163, 165, 166, 167, 168, 169, 171, 172, 174, 175, 177, 178, 179, 180, 181, 182, 184, 185, 186, 187, 190, 191, 192, 193, 196, 198, 200, 201, 202, 203, 205, 206, 207, 209, 210, 212, 214, 216, 218, 221, 222, 223, 224, 225, 226, 227, 229, 230, 231, 234, 236, 237, 238, 239, 240, 241, 244, 246, 247, 248, 249, 253, 254, 255, 261, 262, 266, 270, 273, 274, 275, 276, 277, 278, 280, 282, 283, 285, 287, 290, 291, 292, 294, 295, 296, 297, 299, 300, 301, 302, 304, 305, 306, 307, 308, 309, 310, 311, 314, 316, 317, 318, 

<br>
<br>

<a id='maxShip'></a>
### Greatest Shipping Cost (per client) Heuristic



In [14]:
#Get the greatest shipping cost for each Client


start = time.time()

#choose the lowest cost 
path={}
for i in range(num_customers):
    aux={}
    aux_key=()
    for j in range(num_facilities):

        #empty list for the Client
        if not aux:
            aux[(i,j)] = shipping_cost.get((i,j)) 
            aux_key = ((i,j)) 
            #print(aux)
            #print(aux_key)
        elif not aux[aux_key]>shipping_cost.get((i,j)):
            #print("....")
            #print (i[0],j[0])
            #print(aux[aux_key])
            #print(shipping_cost.get((i[0],j[0])))
            #print ("  ")
            aux.pop(aux_key)
            aux[(i,j)] = shipping_cost.get((i,j))             
            aux_key = ((i,j))
            
        #print(aux_key)
    #print ("xxxxxxx")
    path.update(aux)

#print(path)    

#calculate the setup_cost
facs=[]
for i in path:
    facs.append(i[1])
#print (facs)
facs=list(set(facs))
print (facs)

totalC=0
for w in facs:
    totalC += fixed_cost[w]
    #print(w)

print(totalC)

#calculate the shipping cost
for i in path.values():
    #print(i)
    totalC += i 
    
print(totalC)
print (len(facs))


end = time.time()
print("TIME IS: ",end - start)

[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 13, 14, 15, 16, 17, 22, 24, 25, 27, 29, 30, 31, 32, 34, 35, 38, 40, 42, 43, 45, 47, 48, 49, 51, 52, 54, 56, 57, 59, 61, 62, 63, 64, 66, 68, 71, 74, 75, 80, 81, 82, 83, 87, 89, 90, 92, 93, 94, 95, 97, 99, 100, 102, 105, 109, 110, 112, 114, 116, 117, 118, 120, 121, 123, 124, 125, 126, 128, 133, 134, 137, 140, 143, 144, 145, 146, 147, 148, 151, 152, 153, 154, 155, 157, 158, 159, 160, 165, 166, 167, 169, 170, 172, 173, 174, 175, 178, 181, 182, 184, 187, 190, 191, 192, 193, 195, 196, 199, 200, 203, 206, 207, 209, 211, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 224, 225, 226, 230, 231, 232, 233, 236, 237, 238, 240, 241, 242, 243, 244, 245, 246, 248, 255, 256, 259, 260, 261, 262, 265, 266, 267, 268, 269, 270, 271, 273, 274, 275, 276, 277, 278, 280, 281, 284, 285, 288, 290, 291, 292, 293, 294, 296, 297, 298, 302, 303, 304, 305, 307, 308, 309, 312, 313, 314, 316, 318, 320, 321, 322, 324, 326, 327, 329, 330, 331, 333, 337, 338, 339, 340, 342, 343, 345, 346,

<a id='lowShipFix'></a>
### Lowest Shipping & Fixed Costs Heuristic



In [15]:
#Get the lowest shipping cost for each Client


start = time.time()

#choose the lowest cost 
path={}
for i in range(num_customers):
    aux={}
    aux_key=()
    for j in range(num_facilities):

        #empty list for the Client
        if not aux:
            aux[(i,j)] = shipping_cost.get((i,j)) + fixed_cost[j]
            aux_key = ((i,j)) 
            #print(aux)
            #print(aux_key)
        elif aux[aux_key]>(shipping_cost.get((i,j)) + fixed_cost[j]):
            #print("....")
            #print (i[0],j[0])
            #print(aux[aux_key])
            #print(shipping_cost.get((i[0],j[0])))
            #print ("  ")
            aux.pop(aux_key)
            aux[(i,j)] = shipping_cost.get((i,j))+ fixed_cost[j]             
            aux_key = ((i,j))
            
        #print(aux_key)
    #print ("xxxxxxx")
    path.update(aux)

#print(path)    

#calculate the setup_cost
facs=[]
for i in path:
    facs.append(i[1])
#print (facs)
facs=list(set(facs))
print (facs)

totalC=0
for w in facs:
    totalC += fixed_cost[w]
    #print(w)

print(totalC)

#calculate the shipping cost
for i in path.values():
    #print(i)
    totalC += i 
    
print(totalC)
print (len(facs))


end = time.time()
print("TIME IS: ",end - start)

[1230]
400
817376.9497109994
1
TIME IS:  6.3576109409332275


<a id='maxShipFix'></a>
### Greatest Shipping & Fixed Costs Heuristic



In [16]:
#Get the lowest shipping cost for each Client


start = time.time()

#choose the lowest cost 
path={}
for i in range(num_customers):
    aux={}
    aux_key=()
    for j in range(num_facilities):

        #empty list for the Client
        if not aux:
            aux[(i,j)] = shipping_cost.get((i,j)) + fixed_cost[j]
            aux_key = ((i,j)) 
            #print(aux)
            #print(aux_key)
        elif not aux[aux_key]>(shipping_cost.get((i,j)) + fixed_cost[j]):
            #print("....")
            #print (i[0],j[0])
            #print(aux[aux_key])
            #print(shipping_cost.get((i[0],j[0])))
            #print ("  ")
            aux.pop(aux_key)
            aux[(i,j)] = shipping_cost.get((i,j))+ fixed_cost[j]             
            aux_key = ((i,j))
            
        #print(aux_key)
    #print ("xxxxxxx")
    path.update(aux)

#print(path)    

#calculate the setup_cost
facs=[]
for i in path:
    facs.append(i[1])
#print (facs)
facs=list(set(facs))
print (facs)

totalC=0
for w in facs:
    totalC += fixed_cost[w]
    #print(w)

print(totalC)

#calculate the shipping cost
for i in path.values():
    #print(i)
    totalC += i 
    
print(totalC)
print (len(facs))


end = time.time()
print("TIME IS: ",end - start)

[1825]
999254
1999523826.226026
1
TIME IS:  7.247744083404541
